# **Emotion Detection Using CNN**

**Import Libraries**

In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.regularizers import l2
from keras.models import load_model


**Load Datasets**

In [2]:
TRAIN_DIR = "./images/train"
TEST_DIR = "./images/validation"


In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)


angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)


                                  image     label
0            ./images/train\angry\0.jpg     angry
1            ./images/train\angry\1.jpg     angry
2           ./images/train\angry\10.jpg     angry
3        ./images/train\angry\10002.jpg     angry
4        ./images/train\angry\10016.jpg     angry
...                                 ...       ...
28816  ./images/train\surprise\9969.jpg  surprise
28817  ./images/train\surprise\9985.jpg  surprise
28818  ./images/train\surprise\9990.jpg  surprise
28819  ./images/train\surprise\9992.jpg  surprise
28820  ./images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)


angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])


                                      image     label
0       ./images/validation\angry\10052.jpg     angry
1       ./images/validation\angry\10065.jpg     angry
2       ./images/validation\angry\10079.jpg     angry
3       ./images/validation\angry\10095.jpg     angry
4       ./images/validation\angry\10121.jpg     angry
...                                     ...       ...
7061  ./images/validation\surprise\9806.jpg  surprise
7062  ./images/validation\surprise\9830.jpg  surprise
7063  ./images/validation\surprise\9853.jpg  surprise
7064  ./images/validation\surprise\9878.jpg  surprise
7065   ./images/validation\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         ./images/validation\angry\10052.jpg
1         ./images/validation\angry\10065.jpg
2         ./images/validation\angry\10079.jpg
3         ./images/validation\angry\10095.jpg
4         ./images/validation\angry\10121.jpg
                        ...                  
7061    ./images/validation\surprise\9806.jpg
7062 

**Preprocess Data**

In [8]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [9]:
train_features = extract_features(train['image'])


  0%|          | 0/28821 [00:00<?, ?it/s]

c:\Users\amamo\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [10]:
test_features = extract_features(test['image'])


  0%|          | 0/7066 [00:00<?, ?it/s]

In [11]:
x_train = train_features/255.0
x_test = test_features/255.0


In [12]:
le = LabelEncoder()
le.fit(train['label'])


LabelEncoder()

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])


In [14]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)


**Define Callbacks**

In [15]:
# Define callbacks
# Use .keras for checkpointing to avoid the error
checkpoint = ModelCheckpoint("./model.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_delta=0.0001)

callbacks_list = [checkpoint, early_stopping, reduce_learningrate]


**Define Model**

In [16]:
# Define your model
model = Sequential()

# Convolutional layers with Batch Normalization
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers with L2 Regularization
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation='softmax'))


c:\Users\amamo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Compile Model**

In [17]:
# Compile the model with the Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


**Train Model**

In [19]:
# Fit the model
model.fit(x=x_train, y=y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks=callbacks_list)


Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 946ms/step - accuracy: 0.2868 - loss: 2.4987
Epoch 1: val_accuracy improved from 0.28941 to 0.36994, saving model to ./model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 453s 1s/step - accuracy: 0.2868 - loss: 2.4986 - val_accuracy: 0.3699 - val_loss: 2.3441 - learning_rate: 1.0000e-04
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 950ms/step - accuracy: 0.3307 - loss: 2.3868
Epoch 2: val_accuracy improved from 0.36994 to 0.39924, saving model to ./model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 458s 1s/step - accuracy: 0.3307 - loss: 2.3867 - val_accuracy: 0.3992 - val_loss: 2.2296 - learning_rate: 1.0000e-04
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 953ms/step - accuracy: 0.3697 - loss: 2.2737
Epoch 3: val_accuracy improved from 0.39924 to 0.42669, saving model to ./model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 452s 1s/step - accuracy: 0.3697 - loss: 2.2737 - val_accuracy: 0.4267 - val_loss: 2.1388 - learning_rate: 1.0000e-04
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━

**Load Model and Weights**

In [20]:
model = load_model("./model.keras")


In [21]:
model.save_weights("./emotiondetector_weights.weights.h5")


In [22]:
model.load_weights("./emotiondetector_weights.weights.h5")
